In [ ]:
# Läser in paket 
# Import needed libraries
import tensorflow as tf


import os
import glob
import cv2

import numpy as np

# from tensorflow import keras
# from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPooling2D,Dropout, Input, Concatenate, Lambda, GlobalAveragePooling2D
from tensorflow.keras.utils  import to_categorical

#print('Keras version:',tf.keras.__version__)

# Helper libraries

import sklearn
from   sklearn.model_selection import train_test_split

# Matlab plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#earlystopping
from keras.callbacks import EarlyStopping

# importerar funktioner för att skapa förväxlingsmatriser
from sklearn.metrics import   ConfusionMatrixDisplay, confusion_matrix


import keras_tuner

# Baysian tuner för att hitta bra hyperparametrar
from keras_tuner.tuners import BayesianOptimization



# Läser in 2 färdiga modeller
from tensorflow.keras.models import Model
from tensorflow.keras.applications import  VGG16
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2


from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Tomma listor som ska fyllas med bilder och klasser
train_images = []
train_labels = [] 

for directory_path in glob.glob("/kaggle/input/fruit-recognition/train/train/*"):
    label = directory_path.split("/")[-1] 
    #print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

In [ ]:
img_dir="/kaggle/input/fruit-recognition/train/train/"
# Plockar ut antal klasser
classes = []
for i in os.listdir(img_dir):
    classes.append(i)

num_classes = len(classes)
print('Klasserna i datamaterialet är :',classes)

In [ ]:
# Normalize the images.
train_images = (train_images - np.min(train_images)) / (np.max(train_images) - np.min(train_images))

In [ ]:

# Kod för att printa alla frukter med tillhörande namn
fig = plt.figure(figsize=(5, 15)) # Sätter storlek och räknare 
k = 0
for frukt in classes: 
    for file in os.listdir(img_dir+"/"+frukt)[0:1]: 
        img=mpimg.imread(img_dir+"/"+frukt+"/"+file)
        #plt.figure()
        k=k+1
        plt.subplot(11,3,k) 
        plt.title(frukt)
        plt.imshow(img)
plt.tight_layout() # ändrar avståndet mellan bilderna

Bilderna är av olika kvalitet, vissa är lätta att urskilja med ögat men andra är relativt svåra, detta kommer möjligtvis göra det lite svårare för en modell att lyckas hitta mönster som skiljer dem åt. Till exempel är blåbär och blåa vindruvor väldigt lika för bilderna som printats ovan.

In [ ]:
# Delar upp träningsdata till valideringsmängd , där 60% är träning och 40% validering.

train_images = np.array(train_images)
train_labels = np.array(train_labels)

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.40)


In [ ]:
val_images, test_images, val_labels, test_labels = train_test_split(val_images, val_labels, test_size=0.25)


In [ ]:
# Kollar så att det blir 32x32x3
print(train_images.shape)
print(val_images.shape)
print(test_images.shape)


I träningsdatat finns 10112  i träningsmängden, 5056 i valideringsmängden och 1686 i testmängden, bilderna är 100X100.

In [ ]:
# Ändrar så datat till de minsta dimensionerna som fungerar på de inlästa modellerna
train_images=tf.image.resize_with_pad(train_images, 72, 72, antialias=False)
val_images=tf.image.resize_with_pad(val_images, 72, 72, antialias=False)
test_images=tf.image.resize_with_pad(test_images, 72, 72, antialias=False)

print(train_images.shape)
print(val_images.shape)
print(test_images.shape)

Stoleken på bilderna har minskats för att ta upp mindre storlek. 

In [ ]:
# Printar fördelningen för klasserna inom dataseten

train_label_counts = {}
for label in train_labels:
    if label in train_label_counts:
        train_label_counts[label] += 1
    else:
        train_label_counts[label] = 1

val_label_counts = {}
for label in val_labels:
    if label in val_label_counts:
        val_label_counts[label] += 1
    else:
        val_label_counts[label] = 1

# Sorterar klasserna 
train_label_counts = dict(sorted(train_label_counts.items()))
val_label_counts = dict(sorted(val_label_counts.items()))
        
# Skapar plotsen
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

# Plot train_label_counts
ax[0].bar(train_label_counts.keys(), train_label_counts.values())
ax[0].set_title('Träningsfördelning mellan klasserna')
ax[0].set_ylabel('Antal')

ax[0].tick_params(axis='x', rotation=45) 
ax[0].margins(0)

# Plot val_label_counts
ax[1].bar(val_label_counts.keys(), val_label_counts.values())
ax[1].set_title('Valideringssfördelning mellan klasserna')
ax[1].set_ylabel('Antal')

plt.xticks(rotation=45)

plt.tight_layout()
plt.margins(0)

plt.show()

Fördelningen är relativt jämn mellan klasserna så det är förhopningsvis inga problem med att en modell kommer prediktera skevt pga fördelningen. 

In [ ]:
input_shape = train_images[0].shape # Sätter en input shape som ska användas vid modellering

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Läser in encodern
label_encoder = LabelEncoder()

# Kodar om klasserna till siffror
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)

In [ ]:
# VGG16 från keras
vgg_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights="imagenet", input_shape=input_shape,classes=num_classes)

# Tar alla lager fram till block3_pool från VGG16 och fryser deras vikter med trainable=FALSE

# Tar alla lager fram till block3_pool från VGG16 och fryser deras vikter med trainable=FALSE
# Skapar en ny modell som kopierar vgg_modell fram till lager 'block3_pool'
inputs = vgg_model.inputs
my_model = vgg_model.get_layer('block3_pool').output 

# Fryser alla lagren i modellen
#for layer in my_model.layers:
#  layer.trainable = False

# Lägger till egna lager som ska tränas
my_model = tf.keras.layers.BatchNormalization(axis=-1)(my_model) # normaliserar inputs
my_model = Conv2D(filters=96, 
                        kernel_size=5,
                        activation='relu',
                        padding='same')(my_model)

my_model=(tf.keras.layers.Flatten())(my_model)  # Ser till så lagren är 'full connected'
my_model=(tf.keras.layers.Dense(64,activation='relu'))(my_model)  # tagit bort ett lager och minskat filter /noder i dom jag lagt in som är kvar
outputs=(Dense(units=num_classes, activation='softmax'))(my_model) #sista lagret 
my_model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

# Gör så att det går att träna lagren
for layer in my_model.layers:
  layer.trainable = True

# Freeze the layers
for layer in my_model.layers[:-6]:
    layer.trainable = False


my_model.summary()



In [ ]:
my_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # ändrat inlärningstakten
  loss='categorical_crossentropy',
  metrics=['categorical_accuracy'],
  
)

earlystop= EarlyStopping(monitor='val_loss', patience=3, restore_best_weights = True) # lägger in earlystopping

epochs = 70 # Antal epoker som kan köras
batch_size = 96 # batchstoleken 

# Train the model.
history = my_model.fit(
  train_images, to_categorical(train_labels),
  epochs=epochs,
  batch_size=batch_size,
  verbose = 1,
  validation_data=(val_images,to_categorical(val_labels)),
  callbacks=[earlystop] # Lägger in earlystopping i träningen
)



In [ ]:

# Evaluate the model.
test_loss, test_acc = my_model.evaluate(test_images,to_categorical(test_labels))
print('Test accuracy: %.3f' % test_acc)